In [6]:
import syft as sy
from syft.core.node.common.node_service.tff.tff_messages import TFFMessageWithReply
from syft.core.node.common.node_service.ping.ping_messages import PingMessageWithReply
from syft.grid import GridURL
from syft.core.node.common.node_service.simple.simple_messages import NodeRunnableMessageWithReply
from syft.core.node.common.node_service.get_repr.get_repr_messages import GetReprMessage

import os, sys

import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

# np.random.seed(0)

In [51]:
domain = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into node... done!


In [52]:
domain.datasets

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'metadatas', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.dataset_manager.dataset_manager_messages.GetDatasetsResponse'>
['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module_

Idx,Name,Description,Assets,Id
[0],MedNIST Data,"The MedNIST dataset was gathered from several sets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license. Label Count: 6 Label Mapping: {""AbdomenCT"": 0, ""BreastMRI"": 1, ""CXR"": 2, ""ChestCT"": 3, ""Hand"": 4, ""HeadCT"": 5} Image Dimensions: (64, 64) Total Images: 58954","[""images""] -> [""labels""] ->",03824e77-3d62-426a-bdea-e836ba210c2b
[1],Little MedNIST,Small dataset for TFF testing,"[""images""] -> [""labels""] ->",7e7c4413-796b-46eb-9872-2ebcbc98b6ec


In [4]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(64*64,)),
      tf.keras.layers.Dense(6, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])


In [5]:
model = create_keras_model()

input_spec = [tf.TensorSpec(shape=(1,64*64), dtype=tf.float32, name=None), 
              tf.TensorSpec(shape=(1,1), dtype=tf.int32, name=None)]

functional_model = tff.learning.models.functional_model_from_keras(keras_model=model, 
                                                                   loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
                                                                   input_spec=input_spec)
print(functional_model.input_spec[0])
tff.learning.models.save_functional_model(functional_model=functional_model, path='tmp')

import shutil
shutil.make_archive('archive', 'zip', 'tmp')


TensorSpec(shape=(1, 4096), dtype=tf.float32, name=None)


INFO:tensorflow:Assets written to: tmp/assets


INFO:tensorflow:Assets written to: tmp/assets


'/home/teo/PySyft/notebooks/PySyTFF/archive.zip'

In [35]:
domain.datasets[1].id

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'metadatas', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.dataset_manager.dataset_manager_messages.GetDatasetsResponse'>


'7e7c4413-796b-46eb-9872-2ebcbc98b6ec'

In [58]:
params = {
    'rounds': 5,
    'dataset_id': domain.datasets[1].id,
    'no_clients': 10,
    'OUTPUT_DIR': 'some_dir',
    'noise_multiplier': 0.05,
    'clients_per_round': 2
    }
with open('archive.zip', 'rb') as f:
    model_bytes = f.read()

# params = {}
msg = TFFMessageWithReply(id_dataset="id_dataset", params=params, model_bytes=model_bytes)
reply_msg = domain.send_immediate_msg_with_reply(msg)


['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'metadatas', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.dataset_manager.dataset_manager_messages.GetDatasetsResponse'>


[2022-06-21T18:58:03.343005+0300][CRITICAL][logger]][267011] UnknownPrivateException has been triggered.


['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'exception_msg', 'exception_type', 'get_protobuf_schema', 'icon', 'id', 'msg_id_causing_exception', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.action.exception_action.ExceptionMessage'>


UnknownPrivateException: UnknownPrivateException has been triggered.

In [1]:
# download MedNIST.pkl
if not os.path.exists("./MedNIST.pkl"):
    os.system('curl -O "https://media.githubusercontent.com/media/shubham3121/datasets/main/MedNIST/MedNIST.pkl"')
else:
    print("MedNIST already downloaded")


MedNIST already downloaded


In [9]:
# Helper Methods

import os
import json
import pandas as pd
from PIL import Image
from enum import Enum
from collections import defaultdict
import numpy as np
from syft.core.adp.data_subject_list import DataSubjectList


def get_label_mapping():
    # the data uses the following mapping
    mapping = {
        "AbdomenCT": 0, 
        "BreastMRI": 1, 
        "CXR": 2, 
        "ChestCT": 3, 
        "Hand": 4, 
        "HeadCT": 5
    }
    return mapping

df = pd.read_pickle("./MedNIST.pkl")
mapping = get_label_mapping()

total_num = df.shape[0]
print("Columns:", df.columns)
print("Total Images:", total_num)
print("Label Mapping", mapping)

Columns: Index(['patient_id', 'image', 'label'], dtype='object')
Total Images: 58954
Label Mapping {'AbdomenCT': 0, 'BreastMRI': 1, 'CXR': 2, 'ChestCT': 3, 'Hand': 4, 'HeadCT': 5}


In [26]:
data_subjects = DataSubjectList.from_series(df['patient_id'][:1000])

images = df['image'][:1000]
images = np.dstack(images.values).astype(np.int64)
images = np.rollaxis(images,-1)

In [27]:
labels = df['label'][:1000].to_numpy().astype("int64")
# labels = labels[:1000]

In [28]:

# run this cell
train_image_data = sy.Tensor(images).annotated_with_dp_metadata(
    min_val=0, max_val=255, data_subjects=data_subjects
)
train_label_data = sy.Tensor(labels).annotated_with_dp_metadata(
    min_val=0, max_val=5, data_subjects=data_subjects
)

Tensor annotated with DP Metadata
Tensor annotated with DP Metadata


In [29]:
train_image_data.child.child.decode()

array([[[101., 101., 101., ..., 101., 101., 101.],
        [101., 101., 101., ..., 101., 101., 101.],
        [101., 101., 101., ..., 101., 101., 101.],
        ...,
        [102., 101.,  99., ..., 100., 102., 103.],
        [104., 103., 102., ..., 100., 102., 103.],
        [ 98.,  99., 100., ..., 101., 100., 101.]],

       [[101., 101., 101., ..., 101., 101., 101.],
        [101., 101., 101., ..., 101., 101., 101.],
        [101., 101., 101., ..., 101., 101., 101.],
        ...,
        [101., 101., 101., ..., 101., 101., 101.],
        [101., 101., 101., ..., 101., 101., 101.],
        [101., 101., 101., ..., 101., 101., 101.]],

       [[101., 101., 101., ..., 101., 101., 101.],
        [101., 101., 101., ..., 101., 101., 101.],
        [101., 101., 101., ..., 101., 101., 101.],
        ...,
        [ 99., 110., 118., ..., 101., 100.,  98.],
        [ 94., 105., 118., ..., 103., 103., 103.],
        [103.,  99.,  92., ...,  96.,  98.,  99.]],

       ...,

       [[101., 101., 101

In [31]:
domain.load_dataset(
    name='Little MedNIST',
    assets={
        'images': train_image_data,
        "labels": train_label_data
    },
    description="Small dataset for TFF testing"
)

Loading dataset... checking asset types...                              ['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'content', 'get_protobuf_schema', 'icon', 'id', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.node_setup.node_setup_messages.GetSetUpResponse'>
Loading dataset... uploading...🚀                        ['Reply', 'Request', '_DomainMessageRegistry__domain_message_registry', '__annotations__', '__attr_allowlist__', '__class__', '__class_getitem

Uploading `images`: 100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 37.76it/s]


['Reply', 'Request', '_DomainMessageRegistry__domain_message_registry', '__annotations__', '__attr_allowlist__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__serde_overrides__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'check_permissions', 'class_name', 'get_permissions', 'get_protobuf_schema', 'get_registered_messages', 'icon', 'id', 'kwargs', 'payload', 'post_init', 'pprint', 'reply', 'reply_payload_type', 'reply_to', 'repr_short', 'request_payload_type', 'run', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.upload_service.upload_service_messages.UploadDataComp

Uploading `labels`: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 247.50it/s]


['Reply', 'Request', '_DomainMessageRegistry__domain_message_registry', '__annotations__', '__attr_allowlist__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__serde_overrides__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'check_permissions', 'class_name', 'get_permissions', 'get_protobuf_schema', 'get_registered_messages', 'icon', 'id', 'kwargs', 'payload', 'post_init', 'pprint', 'reply', 'reply_payload_type', 'reply_to', 'repr_short', 'request_payload_type', 'run', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.upload_service.upload_service_messages.UploadDataComp

In [32]:
domain.datasets

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'get_protobuf_schema', 'icon', 'id', 'metadatas', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.dataset_manager.dataset_manager_messages.GetDatasetsResponse'>
['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module_

Idx,Name,Description,Assets,Id
[0],MedNIST Data,"The MedNIST dataset was gathered from several sets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license. Label Count: 6 Label Mapping: {""AbdomenCT"": 0, ""BreastMRI"": 1, ""CXR"": 2, ""ChestCT"": 3, ""Hand"": 4, ""HeadCT"": 5} Image Dimensions: (64, 64) Total Images: 58954","[""images""] -> [""labels""] ->",03824e77-3d62-426a-bdea-e836ba210c2b
[1],Little MedNIST,Small dataset for TFF testing,"[""images""] -> [""labels""] ->",7e7c4413-796b-46eb-9872-2ebcbc98b6ec


In [4]:
domain_cleint = sy.login(url='20.237.150.216',email="info@openmined.org",password="changethis")


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to 20.237.150.216... done! 	 Logging into gifted_song... done!

**Warning**: The syft version on your system and the node are different.
Version on your system: 0.7.0-beta.19
Version on the node: 0.7.0-beta.23



In [17]:
ips = ['20.237.150.216']
details = []

for ip in ips:
    domain_client = sy.login(url='20.237.150.216',email="info@openmined.org",password="changethis")

['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__final__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_id', '_is_protocol', '_object2proto', '_proto2object', 'address', 'class_name', 'content', 'get_protobuf_schema', 'icon', 'id', 'post_init', 'pprint', 'repr_short', 'sign', 'signed_type']
<class 'syft.core.node.common.node_service.user_manager.user_messages.GetUsersResponse'>
{'id': 2, 'email': 'sam@stargate.net', 'name': 'Sam Carter', 'budget': 9999.0, 'verify_key': '1782d56590436d38a648becb0d2ad7872c1cfd353e8f21335b693f8273bc7caa', 'role': 'Data Scientist', 'added_by': 'Jane Doe', 'website': '', 'institution': '', 'daa_pdf': 1, 'created_at': '